In [ ]:
!pip install fsspec

     |████████████████████████████████| 132 kB 22.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:

df = pd.read_csv("heart.csv")

df

,Age,Sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,DV
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2,1
300,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3,1
301,57,0,2,130,236,0,2,174,0,0.0,2,1,3,1,1


In [ ]:
# Missing Value Imputation
df.ca.loc[df.ca=='?']     = '0' 
df.thal.loc[df.thal=='?'] = '3'

# Remove Outlier
df = df[df.chol<500][df.oldpeak<5]

# Remove Unnecessary Column
df.drop(columns = 'num', axis = 1, inplace = True)

# One-Hot Encoding
df = pd.get_dummies(df, columns = ['Sex','cp','fbs','restecg','exang','slope','ca','thal'], drop_first = True)

# Get features and target variables
target=''
target, feature_list = 'DV', [i for i in df.columns if i not in target]

# Print list of features and target variable names
print('Feature List\n',feature_list, '\n\nTarget = ',target)

Feature List
 ['Age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'DV', 'Sex_1', 'cp_2', 'cp_3', 'cp_4', 'fbs_1', 'restecg_1', 'restecg_2', 'exang_1', 'slope_2', 'slope_3', 'ca_1', 'ca_2', 'ca_3', 'thal_6', 'thal_7'] 

Target =  DV


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
def init_population(n,c):
    return np.array([[math.ceil(e) for e in pop] for pop in (np.random.rand(n,c)-0.5)]), np.zeros((2,c))-1


In [ ]:
def single_point_crossover(population):
    r,c, n = population.shape[0], population.shape[1], np.random.randint(1,population.shape[1])         
    for i in range(0,r,2):                
        population[i], population[i+1] = np.append(population[i][0:n],population[i+1][n:c]),np.append(population[i+1][0:n],population[i][n:c])        
    return population

def flip_mutation(population):
    return population.max() - population

def random_selection(population):
    r = population.shape[0]
    new_population = population.copy()    
    for i in range(r):        
        new_population[i] = population[np.random.randint(0,r)]
    return new_population

In [ ]:
def get_fitness(data, feature_list, target, population):    
    fitness = []
    for i in range(population.shape[0]):        
        columns = [feature_list[j] for j in range(population.shape[1]) if population[i,j]==1]                    
        fitness.append(predictive_model(data[columns], data[target]))                
    return fitness

def predictive_model(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=7)
    lr = LogisticRegression(solver='liblinear', max_iter=100, random_state=7)
    lr.fit(X_train,y_train)    
    return accuracy_score(y_test, lr.predict(X_test))

In [ ]:
def ga(data, feature_list, target, n, max_iter):
    
    c = len(feature_list)
    
    population, memory = init_population(n,c)
    #population, memory = replace_duplicate(population, memory)    
    
    fitness    = get_fitness(data, feature_list, target, population)    
    
    optimal_value    = max(fitness)
    optimal_solution = population[np.where(fitness==optimal_value)][0]    
    
    for i in range(max_iter):                
        population = random_selection(population)
        population = single_point_crossover(population)                        
        if np.random.rand() < 0.3:
            population = flip_mutation(population)   
        
        #population, memory = replace_duplicate(population, memory)
                
        fitness = get_fitness(data, feature_list, target, population)
                
        if max(fitness) > optimal_value:
            optimal_value    = max(fitness)
            optimal_solution = population[np.where(fitness==optimal_value)][0]                               
        
    return optimal_solution, optimal_value

In [ ]:
# Execute Genetic Algorithm to obtain Important Feature
feature_set, acc_score = ga(df, feature_list, target, 100, 2000)

# Filter Selected Features
feature_set = [feature_list[i] for i in range(len(feature_list)) if feature_set[i]==1]

# Print List of Features
print('Optimal Feature Set\n',feature_set,'\nOptimal Accuracy =', round(acc_score*100), '%')


Optimal Feature Set
 ['chol', 'DV', 'Sex_1', 'cp_2', 'fbs_1', 'exang_1', 'slope_2', 'slope_3', 'ca_2', 'ca_3', 'thal_7'] 
Optimal Accuracy = 100 %


In [ ]:
['Age', 'trestbps', 'oldpeak', 'Sex_1', 'cp_2', 'exang_1', 'slope_3', 'ca_2', 'thal_7'] 

['Age',
 'trestbps',
 'oldpeak',
 'Sex_1',
 'cp_2',
 'exang_1',
 'slope_3',
 'ca_2',
 'thal_7']